<a href="https://colab.research.google.com/github/gangadharpro/Text-Summarization-using-Transformer-Models/blob/main/Untitled11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import transformers
from datasets import Dataset, DatasetDict
import tensorflow as tf
import os
import re
import math

In [47]:
test=pd.read_csv('test.csv')

In [48]:
test.head()

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."


In [49]:
validation=pd.read_csv('validation.csv')

In [51]:
train = pd.read_csv('train.csv') # Skip bad lines and disable quoting


In [50]:
validation.head()

,id,article,highlights
0,61df4979ac5fcc2b71be46ed6fe5a46ce7f071c3,"Sally Forrest, an actress-dancer who graced th...","Sally Forrest, an actress-dancer who graced th..."
1,21c0bd69b7e7df285c3d1b1cf56d4da925980a68,A middle-school teacher in China has inked hun...,Works include pictures of Presidential Palace ...
2,56f340189cd128194b2e7cb8c26bb900e3a848b4,A man convicted of killing the father and sist...,"Iftekhar Murtaza, 29, was convicted a year ago..."
3,00a665151b89a53e5a08a389df8334f4106494c2,Avid rugby fan Prince Harry could barely watch...,Prince Harry in attendance for England's crunc...
4,9f6fbd3c497c4d28879bebebea220884f03eb41a,A Triple M Radio producer has been inundated w...,Nick Slater's colleagues uploaded a picture to...


In [52]:
train.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [53]:
train.shape

(287113, 3)

In [54]:
test.shape

(11490, 3)

In [56]:
validation.shape

(13368, 3)

In [58]:
train.isnull().sum()


,0
id,0
article,0
highlights,0


In [59]:
test.isnull().sum()


,0
id,0
article,0
highlights,0


In [60]:
validation.isnull().sum()


,0
id,0
article,0
highlights,0


### Data Wrangling

In [61]:
article_length = train['article'].apply(lambda x: len(re.findall(r'\w+', x)))
train['article'] = train['article'].apply(lambda x: x.lower())
train['highlights'] = train['highlights'].apply(lambda x: x.lower())
hightlights_length = train['highlights'].apply(lambda x: len(re.findall(r'\w+', x)))

In [62]:
print(f"maximum article length is {article_length.max()}")
print(f"minimum article length is {article_length.min()}")
print(f"average article length is {article_length.mean()}")

print(f"maximum hightlights length is {hightlights_length.max()}")
print(f"minimum hightlights length is {hightlights_length.min()}")
print(f"average hightlights length is {hightlights_length.mean()}")

maximum article length is 2209
minimum article length is 8
average article length is 698.4268702566585
maximum hightlights length is 1245
minimum hightlights length is 3
average hightlights length is 49.535252670551316


In [64]:
test['article'][0]

"Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee.\xa0'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting for sp

In [66]:
test.shape

(11490, 3)

In [68]:
article_length_test = test['article'].apply(lambda x: len(re.findall(r'\w+', x)))
test['article'] = test['article'].apply(lambda x: x.lower())
test['highlights'] = test['highlights'].apply(lambda x: x.lower())
hightlights_length_test = test['highlights'].apply(lambda x: len(re.findall(r'\w+', x)))

In [69]:
print(f"maximum article length is {article_length_test.max()}")
print(f"minimum article length is {article_length_test.min()}")
print(f"average article length is {article_length_test.mean()}")

print(f"maximum hightlights length is {hightlights_length_test.max()}")
print(f"minimum hightlights length is {hightlights_length_test.min()}")
print(f"average hightlights length is {hightlights_length_test.mean()}")

maximum article length is 1936
minimum article length is 56
average article length is 694.72863359443
maximum hightlights length is 524
minimum hightlights length is 8
average hightlights length is 52.900870322019145


In [70]:
f_ = filter(lambda x: True if x > 1800 else False, article_length)
print(f'no of article with length greater than 1800 is {len([i for i in f_])}')

h_ = filter(lambda x: True if x > 500 else False, article_length)
print(f'no of highlight with length greater than 500 is {len([i for i in h_])}')

no of article with length greater than 1800 is 565
no of highlight with length greater than 500 is 195321


In [72]:
article_length_validation = validation['article'].apply(lambda x: len(re.findall(r'\w+', x)))
validation['article'] = validation['article'].apply(lambda x: x.lower())
validation['highlights'] = validation['highlights'].apply(lambda x: x.lower())
hightlights_length_validation = validation['highlights'].apply(lambda x: len(re.findall(r'\w+', x)))

In [73]:
print(f"maximum article length is {article_length_validation.max()}")
print(f"minimum article length is {article_length_validation.min()}")
print(f"average article length is {article_length_validation.mean()}")

print(f"maximum hightlights length is {hightlights_length_validation.max()}")
print(f"minimum hightlights length is {hightlights_length_validation.min()}")
print(f"average hightlights length is {hightlights_length_validation.mean()}")

maximum article length is 1901
minimum article length is 43
average article length is 686.9938659485338
maximum hightlights length is 1476
minimum hightlights length is 9
average hightlights length is 55.69367145421903
